# Fussing for Clancy

In [2]:
# imports
import pandas
from importlib import resources
import os
from frb.frb import FRB

from astropath.priors import load_std_priors
from astropath.path  import PATH

# Load

In [3]:
my_frb = FRB.by_name('FRB20181112A')

In [4]:
ppath = os.path.join(resources.files('frb'), 'data', 'Galaxies', 'PATH')
pfile = os.path.join(ppath, f'{my_frb.frb_name}_PATH.csv')
ptbl = pandas.read_csv(pfile)

In [5]:
ptbl.head()

,Unnamed: 0,label,xcentroid,ycentroid,sky_centroid,bbox_xmin,bbox_xmax,bbox_ymin,bbox_ymax,area,...,separation,ang_size,P_c,Sigma_m,mag,P_O,p_xO,P_Ox,P_Ux,P_U
0,7,8,56.991932,58.993064,NaN,51,63,53,67,123.0,...,0.428627,0.666238,0.061913,0.000579,21.485312,0.078456,1.773427e-02,9.273796e-01,0.0,0.0
1,8,9,48.844482,77.801460,NaN,38,61,65,90,436.0,...,5.450290,1.057988,0.011065,0.000053,19.098769,0.864243,1.257059e-04,7.241203e-02,0.0,0.0
2,3,4,59.330280,31.046651,NaN,54,64,26,36,84.0,...,6.920492,0.574254,0.214769,0.000934,22.003945,0.048651,6.410474e-06,2.078764e-04,0.0,0.0
3,4,5,73.942374,33.956103,NaN,72,76,32,36,19.0,...,7.290851,0.317543,0.761111,0.005256,24.036615,0.008649,8.714528e-08,5.023954e-07,0.0,0.0


# Priors

In [6]:
# Load prior
priors = load_std_priors()
prior = priors['adopted'] # Default

In [7]:
theta_new = dict(method='exp', 
    max=priors['adopted']['theta']['max'], 
    scale=0.5)
prior['theta'] = theta_new

# Candidates

In [8]:
candidates = ptbl[['ang_size', 'mag', 'ra', 'dec', 'separation']]

In [9]:
candidates.head()

,ang_size,mag,ra,dec,separation
0,0.666238,21.485312,327.348648,-52.970910,0.428627
1,1.057988,19.098769,327.349594,-52.969594,5.450290
2,0.574254,22.003945,327.348376,-52.972866,6.920492
3,0.317543,24.036615,327.346678,-52.972663,7.290851


In [10]:
candidates.mag.values

array([21.48531197, 19.09876882, 22.00394499, 24.03661513])

# PATH time

In [11]:
path_181112 = PATH()
path_181112.init_candidates(candidates.ra.values,
                         candidates.dec.values,
                         candidates.ang_size.values,
                         mag=candidates.mag.values)
path_181112.frb = my_frb

## Localization

In [12]:
frb_eellipse = dict(a=my_frb.sig_a,
                    b=my_frb.sig_b,
                    theta=my_frb.eellipse['theta'])

In [13]:
path_181112.init_localization('eellipse', 
                            center_coord=path_181112.frb.coord,
                            eellipse=frb_eellipse)

## Priors

In [14]:
path_181112.init_cand_prior('inverse', P_U=prior['U'])
path_181112.init_theta_prior(prior['theta']['method'], 
                            prior['theta']['max'],
                            prior['theta']['scale'])

In [15]:
path_181112.calc_priors()                            

array([0.07845585, 0.86424333, 0.04865148, 0.00864934])

## Posterior

In [16]:
# Calculate p(O_i|x)
debug = True
path_181112.calc_posteriors('fixed', 
                         box_hwidth=10., 
                         max_radius=10., 
                         debug=debug)

(array([9.27379594e-01, 7.24120274e-02, 2.07876354e-04, 5.02395446e-07]),
 np.float64(0.0))

In [20]:
path_181112.candidates.sort_values('P_Ox', ascending=False)

,ra,dec,ang_size,mag,P_O,p_xO,P_Ox,P_Ux
0,327.348648,-52.970910,0.666238,21.485312,0.078456,1.773427e-02,9.273796e-01,0.0
1,327.349594,-52.969594,1.057988,19.098769,0.864243,1.257059e-04,7.241203e-02,0.0
2,327.348376,-52.972866,0.574254,22.003945,0.048651,6.410474e-06,2.078764e-04,0.0
3,327.346678,-52.972663,0.317543,24.036615,0.008649,8.714528e-08,5.023954e-07,0.0
